In [21]:
import pandas as pd

### Load Dataset Promos

In [3]:
df_executed_promos = pd.read_csv('../data/executed_promos.csv', encoding='latin')
df_executed_promos

,CodigoDC,Cliente,Marca,Cupo
0,297601,8410,29,9
1,297601,9337,29,9
2,309843,7404,40,16
3,309874,4017,40,16
4,309910,6316,40,16
...,...,...,...,...
27789,651436,9798,39,20
27790,651588,2397,39,20
27791,651597,4430,40,16
27792,651617,7347,39,20


In [4]:
df_active_promos = pd.read_csv('../data/active_promos.csv', encoding='latin')
df_active_promos

,CodigoDC,Marca,Cupo,Fecha_Desde,Fecha_Hasta,Cliente
0,297601,29,9,2018-08-06,2018-08-30,3213
1,297601,29,9,2018-08-06,2018-08-30,3795
2,297601,29,9,2018-08-06,2018-08-30,11816
3,298856,40,16,2018-08-12,2018-08-19,8444
4,309916,40,16,2018-08-12,2018-08-19,8488
...,...,...,...,...,...,...
422432,688354,40,16,2019-09-25,2019-09-26,14435
422433,688355,29,9,2019-09-25,2019-09-26,320
422434,688355,29,9,2019-09-25,2019-09-26,3117
422435,688355,29,9,2019-09-25,2019-09-26,3503


In [5]:
df_active_promos['CodigoDC_Cliente'] = (df_active_promos['CodigoDC'].astype('str') + df_active_promos['Cliente'].astype('str') + 
                                        df_active_promos['Marca'].astype('str') + df_active_promos['Cupo'].astype('str'))
df_executed_promos['CodigoDC_Cliente'] = (df_executed_promos['CodigoDC'].astype('str') + df_executed_promos['Cliente'].astype('str') +
                                          df_executed_promos['Marca'].astype('str') + df_executed_promos['Cupo'].astype('str'))
df_executed_promos['Is_Executed'] = 1

In [6]:
df_promos = pd.concat([df_active_promos.set_index('CodigoDC_Cliente'), df_executed_promos.set_index('CodigoDC_Cliente')[['Is_Executed']] ], axis=1)
df_promos['Cliente_Marca_Cupo'] = df_promos['Cliente'].astype('str') + df_promos['Marca'].astype('str') + df_promos['Cupo'].astype('str')
df_promos['Is_Executed'] = df_promos['Is_Executed'].fillna(0).astype('int32')
import datetime as dt
df_promos['Fecha_Desde'] = df_promos['Fecha_Desde'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
df_promos['Fecha_Hasta'] = df_promos['Fecha_Hasta'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
df_promos['Duracion_Promocion'] = (df_promos['Fecha_Hasta'] - df_promos['Fecha_Desde'])
df_promos['Duracion_Promocion'] = df_promos['Duracion_Promocion'].apply(lambda x: x.days)
df_promos

,CodigoDC,Marca,Cupo,Fecha_Desde,Fecha_Hasta,Cliente,Is_Executed,Cliente_Marca_Cupo,Duracion_Promocion
2976013213299,297601,29,9,2018-08-06,2018-08-30,3213,0,3213299,24
2976013795299,297601,29,9,2018-08-06,2018-08-30,3795,0,3795299,24
29760111816299,297601,29,9,2018-08-06,2018-08-30,11816,1,11816299,24
29885684444016,298856,40,16,2018-08-12,2018-08-19,8444,1,84444016,7
30991684884016,309916,40,16,2018-08-12,2018-08-19,8488,1,84884016,7
...,...,...,...,...,...,...,...,...,...
688354144354016,688354,40,16,2019-09-25,2019-09-26,14435,0,144354016,1
688355320299,688355,29,9,2019-09-25,2019-09-26,320,0,320299,1
6883553117299,688355,29,9,2019-09-25,2019-09-26,3117,0,3117299,1
6883553503299,688355,29,9,2019-09-25,2019-09-26,3503,0,3503299,1


In [7]:
df_promos.isna().sum()

CodigoDC              0
Marca                 0
Cupo                  0
Fecha_Desde           0
Fecha_Hasta           0
Cliente               0
Is_Executed           0
Cliente_Marca_Cupo    0
Duracion_Promocion    0
dtype: int64

In [8]:
df_promos_final = df_promos[['Cliente_Marca_Cupo', 'Cliente', 'Marca', 'Cupo']].drop_duplicates()
df_promos_final.set_index('Cliente_Marca_Cupo', inplace=True)
List_Is_Executed = df_promos.groupby('Cliente_Marca_Cupo').Is_Executed.apply(lambda x: list(x))
List_CodigoDC = df_promos.groupby('Cliente_Marca_Cupo').CodigoDC.apply(lambda x: list(x))
List_Fecha_Desde = df_promos.groupby('Cliente_Marca_Cupo').Fecha_Desde.apply(lambda x: list(x))
List_Fecha_Hasta= df_promos.groupby('Cliente_Marca_Cupo').Fecha_Hasta.apply(lambda x: list(x))
List_Duracion_Promocion= df_promos.groupby('Cliente_Marca_Cupo').Duracion_Promocion.apply(lambda x: list(x))
df_promos_final=pd.concat([df_promos_final, List_CodigoDC, List_Is_Executed, List_Fecha_Desde, List_Fecha_Hasta, List_Duracion_Promocion], axis=1)
df_promos_final

,Cliente,Marca,Cupo,CodigoDC,Is_Executed,Fecha_Desde,Fecha_Hasta,Duracion_Promocion
3213299,3213,29,9,"[297601, 354404, 619037, 619621, 542054]","[0, 0, 0, 0, 0]","[2018-08-06 00:00:00, 2018-09-03 00:00:00, 201...","[2018-08-30 00:00:00, 2018-09-27 00:00:00, 201...","[24, 24, 5, 3, 3]"
3795299,3795,29,9,"[297601, 354404, 542055, 619684, 644816, 61843...","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[2018-08-06 00:00:00, 2018-09-03 00:00:00, 201...","[2018-08-30 00:00:00, 2018-09-27 00:00:00, 201...","[24, 24, 3, 3, 7, 5, 17, 9, 9]"
11816299,11816,29,9,"[297601, 354404, 646169, 659921, 660481, 542055]","[1, 0, 0, 0, 0, 0]","[2018-08-06 00:00:00, 2018-09-03 00:00:00, 201...","[2018-08-30 00:00:00, 2018-09-27 00:00:00, 201...","[24, 24, 7, 9, 9, 3]"
84444016,8444,40,16,"[298856, 355104, 357820, 360816, 354320, 40554...","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[2018-08-12 00:00:00, 2018-09-05 00:00:00, 201...","[2018-08-19 00:00:00, 2018-09-06 00:00:00, 201...","[7, 1, 4, 13, 1, 6, 9, 0, 9, 5, 8, 3, 3, 5, 13..."
84884016,8488,40,16,"[309916, 355700, 354347, 354783, 374390, 35643...","[1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, ...","[2018-08-12 00:00:00, 2018-09-06 00:00:00, 201...","[2018-08-19 00:00:00, 2018-09-06 00:00:00, 201...","[7, 0, 13, 1, 5, 4, 22, 6, 0, 11, 4, 9, 5, 0, ..."
...,...,...,...,...,...,...,...,...
150403920,15040,39,20,[651597],[0],[2019-08-27 00:00:00],[2019-08-30 00:00:00],[3]
148404016,14840,40,16,[651714],[0],[2019-08-27 00:00:00],[2019-08-30 00:00:00],[3]
1844299,1844,29,9,"[660164, 660709]","[0, 0]","[2019-09-05 00:00:00, 2019-09-05 00:00:00]","[2019-09-14 00:00:00, 2019-09-14 00:00:00]","[9, 9]"
80103920,8010,39,20,[673992],[0],[2019-09-18 00:00:00],[2019-09-29 00:00:00],[11]


In [9]:
df_promos_final.isna().sum()

Cliente               0
Marca                 0
Cupo                  0
CodigoDC              0
Is_Executed           0
Fecha_Desde           0
Fecha_Hasta           0
Duracion_Promocion    0
dtype: int64

### Load Dataset Sales and Clients

In [10]:
df_clients = pd.read_csv('../data/clients_attributes.csv')
df_clients

,Cliente,FechaAltaCliente,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF
0,1,2009-03-03,4,21,12,1,1,1
1,9,2009-03-03,4,21,9,2,3,1
2,15,2009-03-03,4,21,9,2,3,0
3,37,2009-03-03,4,21,16,2,3,1
4,43,2009-03-03,4,21,9,2,3,0
...,...,...,...,...,...,...,...,...
15064,15040,2019-03-07,1,16,9,2,1,0
15065,15043,2019-03-07,1,20,15,2,2,1
15066,15052,2019-03-09,3,7,10,2,2,0
15067,15065,2019-03-16,4,21,9,2,2,0


In [11]:
df_sales = pd.read_csv('../data/sales.csv', encoding='latin') ## no hay clase segmento precio y cupo
### completar los valores faltantes
df_sales['Cupo'] = df_sales['Cupo'].apply(lambda x:  int(x) if str(x)!='nan' else 16 )
df_sales['SegmentoPrecio'] = df_sales['SegmentoPrecio'].apply(lambda x:  int(x) if str(x)!='nan' else 1)
df_sales

,Cliente,Año,Mes,ClaseEnvase,SegmentoPrecio,Marca,Cupo,Nr,Hl,Dcto
0,1,2018,1,2,2,44,10,6352.94,0.03,0.0
1,1,2018,1,2,4,18,16,99130.50,0.45,0.0
2,1,2018,6,2,1,2,6,11605.40,0.07,-2880.0
3,1,2018,7,1,2,37,34,1944.01,0.01,0.0
4,1,2018,8,2,1,2,16,16250.18,0.06,0.0
...,...,...,...,...,...,...,...,...,...,...
1894247,15068,2019,8,1,2,44,32,3777.28,0.01,0.0
1894248,15069,2019,3,1,2,44,32,6295.46,0.02,0.0
1894249,15069,2019,4,1,2,44,32,6295.46,0.02,0.0
1894250,15069,2019,4,2,1,2,6,15741.56,0.08,0.0


In [12]:
#### Analizando Sales_Clients
df_clients_rec = pd.concat([df_clients.set_index('Cliente'),
df_sales.groupby('Cliente').Nr.apply(lambda x:list(x)),
df_sales.groupby('Cliente').Hl.apply(lambda x:list(x)),
df_sales.groupby('Cliente').Dcto.apply(lambda x:list(x)),
df_sales.groupby('Cliente').Año.apply(lambda x:list(x)),
df_sales.groupby('Cliente').Mes.apply(lambda x:list(x)),
df_sales.groupby('Cliente').ClaseEnvase.apply(lambda x:list(x)),
df_sales.groupby('Cliente').SegmentoPrecio.apply(lambda x:list(x)),
df_sales.groupby('Cliente').Marca.apply(lambda x:list(x)),
df_sales.groupby('Cliente').Cupo.apply(lambda x:list(x))], axis=1)
df_clients_rec = df_clients_rec.reset_index().rename(columns = {'index':'Cliente'})
df_clients_rec.rename(columns={'Marca':'ListMarca',	'Cupo':'ListCupo'}, inplace=True)
df_clients_rec.head()

,Cliente,FechaAltaCliente,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr,Hl,Dcto,Año,Mes,ClaseEnvase,SegmentoPrecio,ListMarca,ListCupo
0,1,2009-03-03,4,21,12,1,1,1,"[6352.94, 99130.5, 11605.4, 1944.01, 16250.18,...","[0.03, 0.45, 0.07, 0.01, 0.06, 0.05, 0.05, 0.0...","[0.0, 0.0, -2880.0, 0.0, 0.0, 0.0, -1898.32, 0...","[2018, 2018, 2018, 2018, 2018, 2018, 2018, 201...","[1, 1, 6, 7, 8, 9, 9, 10, 10, 11, 11, 11, 12, ...","[2, 2, 2, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2, 2, ...","[2, 4, 1, 2, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 1, ...","[44, 18, 2, 37, 2, 44, 44, 44, 6, 9, 44, 2, 44...","[10, 16, 6, 34, 16, 30, 32, 32, 12, 25, 29, 16..."
1,2,2009-03-03,4,21,9,2,5,0,"[24277.31, 79984.8, 58244.09, 7724.52, 36714.8...","[0.13, 0.09, 0.18, 0.02, 0.13, 0.36, 0.13, 0.1...","[0.0, -6955.2, -2740.5, 0.0, -5774.27, -6458.3...","[2018, 2018, 2018, 2018, 2018, 2018, 2018, 201...","[1, 1, 2, 2, 3, 3, 3, 4, 4, 5, 6, 7, 7, 8, 9, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, ...","[2, 4, 1, 1, 1, 3, 4, 1, 3, 2, 4, 2, 4, 3, 1, ...","[44, 31, 2, 28, 40, 23, 31, 40, 46, 37, 31, 44...","[29, 8, 26, 25, 22, 26, 8, 22, 23, 34, 8, 33, ..."
2,3,2009-03-03,4,21,9,2,1,0,"[40462.19, 35956.55, 48554.62, 28980.0, 9647.0...","[0.22, 0.12, 0.27, 0.03, 0.03, 0.04, 0.06, 0.1...","[0.0, 0.0, 0.0, 0.0, 0.0, -544.47, 0.0, 0.0, 0...","[2018, 2018, 2018, 2018, 2018, 2018, 2018, 201...","[1, 1, 2, 2, 2, 3, 3, 4, 5, 5, 5, 7, 7, 8, 8, ...","[1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 2, ...","[2, 1, 2, 4, 3, 2, 4, 2, 2, 1, 2, 1, 1, 4, 1, ...","[44, 6, 44, 31, 23, 44, 17, 44, 44, 2, 44, 2, ...","[29, 16, 29, 8, 16, 32, 23, 29, 29, 19, 16, 19..."
3,4,2009-03-03,4,21,18,2,4,1,"[113294.1, 36592.91, 62538.29, 121764.74, 1898...","[0.63, 0.15, 0.26, 0.45, 0.05, 0.3, 0.04, 0.63...","[0.0, -1524.71, -991.06, 0.0, 0.0, 0.0, -1898....","[2018, 2018, 2018, 2018, 2018, 2018, 2018, 201...","[1, 1, 2, 3, 4, 5, 5, 8, 10, 10, 1, 3, 3, 3, 3...","[1, 2, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 37, 4...","[29, 10, 10, 16, 32, 30, 32, 29, 30, 10, 34, 3..."
4,5,2009-03-03,4,21,11,2,1,0,"[42336.0, 566315.63, 461062.5, 763886.73, 9161...","[0.12, 1.87, 1.8, 3.6, 0.3, 0.95, 0.21, 0.83, ...","[0.0, 0.0, -12061.5, -161363.67, 0.0, -19219.4...","[2018, 2018, 2018, 2018, 2018, 2018, 2018, 201...","[1, 2, 2, 3, 4, 5, 6, 6, 6, 1, 1, 2, 2, 4, 4, ...","[1, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 1, 1, ...","[1, 1, 1, 1, 1, 1, 4, 1, 1, 2, 2, 1, 1, 4, 4, ...","[6, 6, 6, 6, 6, 6, 17, 2, 6, 44, 44, 2, 2, 17,...","[25, 16, 19, 19, 16, 16, 23, 19, 19, 29, 30, 1..."


## Unir las bases de Promos, Ventas y Clientes

In [13]:
# Cliente_Sales es la variable que me indica cuantas el cliente ha comprado dicho producto de una respectiva marca y cupo

df_sales_clients = pd.merge(df_sales, df_clients, how = 'left', on = 'Cliente')
df_sales_clients['Cliente_Marca_Cupo'] = df_sales_clients['Cliente'].astype('str') + df_sales_clients['Marca'].astype('str') + df_sales_clients['Cupo'].astype('str')
aux =  df_sales_clients.groupby('Cliente_Marca_Cupo').size()
aux.name = 'Cliente_Sales'
df_promos_final = pd.merge(df_promos_final, aux, how = 'left', left_index=True, right_index=True).fillna(0)
df_promos_final['Cliente_Sales'] = df_promos_final['Cliente_Sales'].astype('int32')
df_promos_final

,Cliente,Marca,Cupo,CodigoDC,Is_Executed,Fecha_Desde,Fecha_Hasta,Duracion_Promocion,Cliente_Sales
3213299,3213,29,9,"[297601, 354404, 619037, 619621, 542054]","[0, 0, 0, 0, 0]","[2018-08-06 00:00:00, 2018-09-03 00:00:00, 201...","[2018-08-30 00:00:00, 2018-09-27 00:00:00, 201...","[24, 24, 5, 3, 3]",2
3795299,3795,29,9,"[297601, 354404, 542055, 619684, 644816, 61843...","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[2018-08-06 00:00:00, 2018-09-03 00:00:00, 201...","[2018-08-30 00:00:00, 2018-09-27 00:00:00, 201...","[24, 24, 3, 3, 7, 5, 17, 9, 9]",4
11816299,11816,29,9,"[297601, 354404, 646169, 659921, 660481, 542055]","[1, 0, 0, 0, 0, 0]","[2018-08-06 00:00:00, 2018-09-03 00:00:00, 201...","[2018-08-30 00:00:00, 2018-09-27 00:00:00, 201...","[24, 24, 7, 9, 9, 3]",1
84444016,8444,40,16,"[298856, 355104, 357820, 360816, 354320, 40554...","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[2018-08-12 00:00:00, 2018-09-05 00:00:00, 201...","[2018-08-19 00:00:00, 2018-09-06 00:00:00, 201...","[7, 1, 4, 13, 1, 6, 9, 0, 9, 5, 8, 3, 3, 5, 13...",15
84884016,8488,40,16,"[309916, 355700, 354347, 354783, 374390, 35643...","[1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, ...","[2018-08-12 00:00:00, 2018-09-06 00:00:00, 201...","[2018-08-19 00:00:00, 2018-09-06 00:00:00, 201...","[7, 0, 13, 1, 5, 4, 22, 6, 0, 11, 4, 9, 5, 0, ...",12
...,...,...,...,...,...,...,...,...,...
150403920,15040,39,20,[651597],[0],[2019-08-27 00:00:00],[2019-08-30 00:00:00],[3],0
148404016,14840,40,16,[651714],[0],[2019-08-27 00:00:00],[2019-08-30 00:00:00],[3],0
1844299,1844,29,9,"[660164, 660709]","[0, 0]","[2019-09-05 00:00:00, 2019-09-05 00:00:00]","[2019-09-14 00:00:00, 2019-09-14 00:00:00]","[9, 9]",3
80103920,8010,39,20,[673992],[0],[2019-09-18 00:00:00],[2019-09-29 00:00:00],[11],0


In [18]:
df_intermediate = pd.merge(df_promos_final, df_clients_rec, how='left', on='Cliente')
df_intermediate['Cliente_Marca_Cupo'] = df_intermediate['Cliente'].astype('str')+df_intermediate['Marca'].astype('str')+df_intermediate['Cupo'].astype('str')
df_intermediate.set_index('Cliente_Marca_Cupo', inplace=True)
df_intermediate.head()

,Cliente,Marca,Cupo,CodigoDC,Is_Executed,Fecha_Desde,Fecha_Hasta,Duracion_Promocion,Cliente_Sales,FechaAltaCliente,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr,Hl,Dcto,Año,Mes,ClaseEnvase,SegmentoPrecio,ListMarca,ListCupo
Cliente_Marca_Cupo,,,,,,,,,,,,,,,,,,,,,,,,,
3213299,3213,29,9,"[297601, 354404, 619037, 619621, 542054]","[0, 0, 0, 0, 0]","[2018-08-06 00:00:00, 2018-09-03 00:00:00, 201...","[2018-08-30 00:00:00, 2018-09-27 00:00:00, 201...","[24, 24, 5, 3, 3]",2,2010-03-29,3,8,15,2,3,1,"[29082.62, 16184.87, 76800.02, 28110.53, 9076....","[0.09, 0.09, 0.25, 0.09, 0.02, 0.03, 0.03, 0.0...","[0.0, 0.0, 0.0, -1333.83, 0.0, 0.0, 0.0, -621....","[2018, 2018, 2018, 2018, 2018, 2018, 2018, 201...","[2, 3, 3, 3, 4, 4, 5, 5, 5, 6, 6, 6, 6, 6, 7, ...","[2, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, ...","[1, 2, 1, 3, 1, 3, 1, 3, 3, 1, 1, 2, 1, 1, 1, ...","[32, 44, 32, 23, 28, 23, 2, 23, 23, 2, 40, 37,...","[17, 29, 17, 16, 25, 26, 26, 26, 16, 26, 22, 3..."
3795299,3795,29,9,"[297601, 354404, 542055, 619684, 644816, 61843...","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[2018-08-06 00:00:00, 2018-09-03 00:00:00, 201...","[2018-08-30 00:00:00, 2018-09-27 00:00:00, 201...","[24, 24, 3, 3, 7, 5, 17, 9, 9]",4,2010-04-26,3,10,6,2,3,1,"[84189.6, 115610.44, 106536.01, 19009.21, 9504...","[0.26, 0.36, 0.13, 0.05, 0.03, 0.05, 0.03, 0.6...","[0.0, 0.0, -9264.0, 0.0, 0.0, -1079.72, -754.7...","[2018, 2018, 2018, 2018, 2018, 2018, 2018, 201...","[1, 1, 2, 3, 3, 4, 4, 5, 6, 6, 7, 9, 12, 1, 2,...","[1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1, ...","[1, 1, 4, 3, 3, 3, 3, 1, 4, 1, 1, 1, 3, 4, 4, ...","[40, 40, 30, 23, 26, 23, 23, 40, 17, 2, 6, 2, ...","[22, 16, 18, 26, 26, 26, 16, 22, 12, 16, 16, 1..."
11816299,11816,29,9,"[297601, 354404, 646169, 659921, 660481, 542055]","[1, 0, 0, 0, 0, 0]","[2018-08-06 00:00:00, 2018-09-03 00:00:00, 201...","[2018-08-30 00:00:00, 2018-09-27 00:00:00, 201...","[24, 24, 7, 9, 9, 3]",1,2017-10-01,3,7,12,2,3,1,"[91605.99, 48655.49, 64183.96, 39308.15, 49778...","[0.33, 0.15, 0.21, 0.12, 0.07, 0.24, 0.09, 1.6...","[0.0, 0.0, 0.0, 0.0, -4328.55, 0.0, 0.0, -2276...","[2018, 2018, 2018, 2018, 2018, 2018, 2018, 201...","[1, 1, 2, 2, 3, 4, 5, 5, 6, 6, 6, 7, 7, 7, 7, ...","[2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, ...","[1, 3, 1, 3, 4, 3, 1, 1, 4, 1, 1, 1, 1, 1, 3, ...","[2, 46, 6, 26, 31, 23, 32, 40, 17, 6, 40, 4, 7...","[16, 16, 16, 16, 8, 16, 17, 16, 12, 16, 16, 16..."
84444016,8444,40,16,"[298856, 355104, 357820, 360816, 354320, 40554...","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[2018-08-12 00:00:00, 2018-09-05 00:00:00, 201...","[2018-08-19 00:00:00, 2018-09-06 00:00:00, 201...","[7, 1, 4, 13, 1, 6, 9, 0, 9, 5, 8, 3, 3, 5, 13...",15,2015-09-02,5,27,18,2,2,1,"[31322.21, 34399.16, 27663.75, 87380.26, 18442...","[0.09, 0.19, 0.11, 0.41, 0.07, 0.5, 0.06, 0.05...","[-316.39, -28121.85, -723.69, -48000.0, -482.4...","[2018, 2018, 2018, 2018, 2018, 2018, 2018, 201...","[1, 1, 1, 1, 2, 2, 2, 3, 3, 4, 4, 4, 4, 5, 5, ...","[1, 1, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 1, 1, ...","[2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, ...","[44, 44, 6, 40, 6, 40, 44, 28, 37, 7, 6, 28, 4...","[32, 33, 19, 16, 19, 16, 16, 30, 16, 25, 19, 1..."
84884016,8488,40,16,"[309916, 355700, 354347, 354783, 374390, 35643...","[1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, ...","[2018-08-12 00:00:00, 2018-09-06 00:00:00, 201...","[2018-08-19 00:00:00, 2018-09-06 00:00:00, 201...","[7, 0, 13, 1, 5, 4, 22, 6, 0, 11, 4, 9, 5, 0, ...",12,2015-09-16,3,10,16,2,2,0,"[84809.55, 39496.79, 20699.98, 68551.66, 16521...","[0.1, 0.13, 0.09, 0.2, 0.06, 0.03, 0.06, 0.09,...","[-7459.2, 0.0, -3652.94, 0.0, 0.0, 0.0, -714.2...","[2018, 2018, 2018, 2018, 2018, 2018, 2018, 201...","[1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 5, 5, 5, ...","[1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, ...","[4, 1, 2, 1, 1, 2, 3, 1, 1, 3, 3, 1, 2, 1, 1, ...","[30, 32, 37, 40, 28, 37, 24, 28, 40, 23, 26, 2...","[18, 17, 16, 22, 16, 16, 16, 16, 19, 16, 16, 1..."


In [19]:
### visualizar si tiene valores faltantes
df_intermediate.isna().sum()

Cliente               0
Marca                 0
Cupo                  0
CodigoDC              0
Is_Executed           0
Fecha_Desde           0
Fecha_Hasta           0
Duracion_Promocion    0
Cliente_Sales         0
FechaAltaCliente      0
Region                0
Gerencia              0
SubCanal              0
TipoPoblacion         0
Estrato               0
EF                    0
Nr                    0
Hl                    0
Dcto                  0
Año                   0
Mes                   0
ClaseEnvase           0
SegmentoPrecio        0
ListMarca             0
ListCupo              0
dtype: int64

In [20]:
df_intermediate.to_pickle('../data/df_intermediate_v3.pkl')